In [22]:
# Test calling the Java Tritech File library
import jpype
import jpype.imports
from jpype import startJVM, shutdownJVM, java, addClassPath, JClass, JInt, JArray

#global _JVM_started
#_JVM_started = 0
try:
    startJVM(convertStrings=False)
except:
    print("Problem starting JVM")
java.lang.System.out.println("hello world")
  



Problem starting JVM


In [ ]:
import numpy
import matplotlib.pyplot as plt
import datetime

jpype.addClassPath("C:\\Users\\dg50\\source\\repos\\TritechFiles\\TritechFilesV1002.jar")
sampleFile = "E:\\GLFYear6\\Trip2\\20251217\\log_2025-12-17-000335.glf"
# create the object that can turn the rectangular image into a fan image
# this only needs to be made once, though if using multiple sonars, it's good
# to have one per sonar. The fan maker holds a lot of translation data in lookup
# tables which will automatically rebuild whenever sonar dimensions change do 
# it gets quite inefficient it two sonars wiht slightly different frame sizes use
# the same one. 
fanMaker = JClass('tritechgemini.imagedata.FanPicksFromData')(4)
str = java.lang.String(sampleFile)
mfc = JClass('tritechgemini.fileio.MultiFileCatalog')()
mfc.catalogFiles(str, java.lang.Boolean(True))
nRec = mfc.getNumRecords()
print('Number of records in file is ', nRec)
# read the first record
aRec = mfc.getFullRecord(0)
#print(aRec);
# get the basic dimensions
nBeam = aRec.getnBeam()
nRange = aRec.getnRange()
maxRange = aRec.getMaxRange()
bearingTable = aRec.getBearingTable()
# get the image data from the record. Remember to get this as short 
imageData = aRec.getShortImageData()
# convert to a numpy array.
pData = numpy.array(imageData)
# reshape the one dimensional array into 2D
gridData = pData.reshape(nRange, nBeam)
# now use other functions in the Java library to create a fan image
fanImage = fanMaker.createFanData(aRec, 1000)
fanData = numpy.array(fanImage.getImageValues()).transpose()
shape = numpy.shape(fanData); # gives height and width
xExtent = fanImage.getMetresPerPixX()*shape[1]
yExtent = fanImage.getMetresPerPixY()*shape[0]

plt.subplot(1,2,1)
plt.imshow(gridData, origin='lower')
plt.subplot(1,2,2)
plt.imshow(fanData, origin='lower', extent=(-xExtent/2, xExtent/2, 0, yExtent))
plt.xlabel('Distance (m)')
dt = datetime.datetime.fromtimestamp(aRec.getRecordTime()/1000.0)
plt.title(dt);

TypeError: No matching overloads found for tritechgemini.fileio.MultiFileCatalog.catalogFiles(java.lang.String,java.lang.Boolean), options are:
	public void tritechgemini.fileio.MultiFileCatalog.catalogFiles(java.lang.String[])


In [ ]:
shutdownJVM()